In [1]:
#Import basic packages
import os
import numpy as np
import pandas as pd
import csv
import regex as re
import datetime as dt 

In [2]:
#import outcome file with org_uuid
main_df = pd.read_csv(R"d:/msc-project/data/final/outcome_final_v1.csv",encoding='utf-8',
                         parse_dates= ['founded_on','seed_date','series_a_date','series_b_date','series_c_date'])
main_processing_df = main_df[['org_uuid']]

In [3]:
main_df.head(3)

,org_uuid,name,status,founded_on,num_funding_rounds,seed_date,series_a_date,first_fund_date,series_b_date,series_c_date,...,went_public_on,closed_on,founded_to_first_fund_years,first_fund_to_series_a_years,first_fund_to_series_b_years,first_fund_to_series_c_years,first_fund_to_acquired_years,first_fund_to_ipo_years,first_fund_to_close_years,outcome
0,eb37f7c8-c036-a915-bb7c-63f445330f66,Ceyba,closed,2000-05-01,2.0,NaT,2000-06-26,2000-06-26,2001-05-31,NaT,...,NaN,2003-08-05,0.2,0.0,0.93,NaN,NaN,NaN,3.1,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,IceFyre Semiconductor,closed,2001-01-01,4.0,NaT,2001-03-12,2001-03-12,2001-08-08,2003-02-18,...,NaN,2005-05-01,0.2,0.0,0.41,1.94,NaN,NaN,4.1,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,ORMvision,closed,2000-12-01,2.0,NaT,2001-05-01,2001-05-01,2001-11-06,NaT,...,NaN,2007-01-01,0.4,0.0,0.52,NaN,NaN,NaN,5.7,0


In [4]:
len(main_processing_df)

28658

In [5]:
#import main organisation csv
org_ori_df = pd.read_csv(R"d:/msc-project/data/pre-processed/organisations_preprocessed.csv",encoding='utf-8',
                         parse_dates= ['founded_on','went_public_on','acquired_on','closed_on'])

In [6]:
org_ori_df.head(3)

,org_uuid,name,legal_name,homepage_url,country_code,state_code,region,city,address,postal_code,...,acquirer_state_code,acquirer_region,acquirer_city,acquisition_type,acquired_on,price_usd,price,price_currency_code,parent_uuid,parent_name
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,NaN,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,...,NY,New York,New York,acquisition,2013-12-16,30000000.0,30000000.0,USD,NaN,NaN
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,"Digg Holdings, LLC",http://www.digg.com,USA,NY,New York,New York,NaN,NaN,...,NY,New York,New York,acquisition,2012-07-12,16000000.0,16000000.0,USD,NaN,NaN
2,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,"Facebook, Inc.",http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [7]:
org_ori_df.columns

Index(['org_uuid', 'name', 'legal_name', 'homepage_url', 'country_code',
       'state_code', 'region', 'city', 'address', 'postal_code', 'status',
       'short_description', 'category_list', 'category_groups_list',
       'num_funding_rounds', 'total_funding_usd', 'total_funding',
       'total_funding_currency_code', 'founded_on', 'last_funding_on',
       'closed_on', 'employee_count', 'email', 'phone', 'facebook_url',
       'linkedin_url', 'twitter_url', 'logo_url', 'primary_role', 'num_exits',
       'description', 'ipo_uuid', 'stock_exchange_symbol', 'stock_symbol',
       'went_public_on', 'share_price_usd', 'share_price',
       'share_price_currency_code', 'valuation_price_usd', 'valuation_price',
       'valuation_price_currency_code', 'money_raised_usd', 'money_raised',
       'money_raised_currency_code', 'acquisition_uuid', 'acquirer_uuid',
       'acquirer_name', 'acquirer_country_code', 'acquirer_state_code',
       'acquirer_region', 'acquirer_city', 'acquisition_type

In [8]:
org_processing = org_ori_df[['org_uuid','category_groups_list']]

In [9]:
org_processing.head(3)

,org_uuid,category_groups_list
0,e1393508-30ea-8a36-3f96-dd3226033abd,"Content and Publishing,Internet Services,Media..."
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,"Internet Services,Media and Entertainment"
2,df662812-7f97-0b43-9d3e-12f64f504fbb,"Apps,Community and Lifestyle,Content and Publi..."


In [10]:
#create a list of unique categories
#categories_list = list(org_processing['category_list'])
#unique_categories = set()

#for category_list in categories_list:
    #category_list = category_list.split(',')
    #for category in category_list:
        #unique_categories.add(category)
        
#unique_categories = list (unique_categories)
#unique_categories.sort()

In [11]:
#assign one hot encode for category_list for each company
#for category in unique_categories:
    #column_name = 'is_'+category.lower().replace(' ','_')
    #one_hot = list()

    #for category_list in categories_list:
        #category_list = category_list.split(',')
        #if category in category_list:
            #one_hot.append(1)
        #else:
            #one_hot.append(0)

    #org_processing[column_name]=one_hot

In [12]:
org_processing.head(3)

,org_uuid,category_groups_list
0,e1393508-30ea-8a36-3f96-dd3226033abd,"Content and Publishing,Internet Services,Media..."
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,"Internet Services,Media and Entertainment"
2,df662812-7f97-0b43-9d3e-12f64f504fbb,"Apps,Community and Lifestyle,Content and Publi..."


In [13]:
#create a list of unique category groups
groups_list = list(org_processing['category_groups_list'])
unique_groups = set()

for group_list in groups_list:
    group_list = group_list.split(',')
    for group in group_list:
        unique_groups.add(group)
        
unique_groups = list (unique_groups)
unique_groups.sort()

In [14]:
len(unique_groups)

47

In [15]:
#assign one hot encode for category_list for each company
for group in unique_groups:
    column_name = 'is_'+group.lower().replace(' ','_')
    one_hot = list()

    for group_list in groups_list:
        group_list = group_list.split(',')
        if group in group_list:
            one_hot.append(1)
        else:
            one_hot.append(0)

    org_processing[column_name]=one_hot

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [16]:
org_processing.head(3)

,org_uuid,category_groups_list,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,...,is_professional_services,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video
0,e1393508-30ea-8a36-3f96-dd3226033abd,"Content and Publishing,Internet Services,Media...",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,"Internet Services,Media and Entertainment",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,df662812-7f97-0b43-9d3e-12f64f504fbb,"Apps,Community and Lifestyle,Content and Publi...",0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [17]:
org_processing.shape

(153048, 49)

In [18]:
org_processing.drop(columns = ['category_groups_list'],inplace=True)

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
org_processing

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_professional_services,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video
0,e1393508-30ea-8a36-3f96-dd3226033abd,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,df662812-7f97-0b43-9d3e-12f64f504fbb,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1
3,60485007-8856-bbac-aa1b-c535c41f5f47,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,4111dc8b-c0df-2d24-ed33-30cd137b3098,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153043,ab278d7c-eb9f-4ed3-8c65-24b7685145a4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153044,e3baa88d-8592-40cc-8ff5-c40254b444e2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
153045,03249b8e-ea2b-4839-a409-232f796f0e62,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
153046,e2760a4a-4645-4e09-a201-3f92807123be,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [20]:
main_processing_df_v1 = pd.merge (main_processing_df,
                                 org_processing,
                                 on='org_uuid',
                                 how='left')
main_processing_df_v1.head(3)

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_professional_services,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [21]:
org_processing = org_ori_df[['org_uuid','founded_on','country_code','state_code']]

In [22]:
org_processing.head(3)

,org_uuid,founded_on,country_code,state_code
0,e1393508-30ea-8a36-3f96-dd3226033abd,2005-06-01,USA,NY
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,2004-10-11,USA,NY
2,df662812-7f97-0b43-9d3e-12f64f504fbb,2004-02-04,USA,CA


In [23]:
org_processing['founded_year'] = org_processing['founded_on'].dt.year

C:\Users\Abdul Rahim Ab Ghani\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
org_processing.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year
0,e1393508-30ea-8a36-3f96-dd3226033abd,2005-06-01,USA,NY,2005
1,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,2004-10-11,USA,NY,2004
2,df662812-7f97-0b43-9d3e-12f64f504fbb,2004-02-04,USA,CA,2004


In [25]:
org_processing = pd.merge(main_processing_df_v1['org_uuid'],
                         org_processing,
                         on='org_uuid',
                         how='left')
org_processing.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year
0,eb37f7c8-c036-a915-bb7c-63f445330f66,2000-05-01,CAN,ON,2000
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,2001-01-01,CAN,ON,2001
2,de462c42-b0a5-c5af-9637-ec426b4e991f,2000-12-01,BEL,NaN,2000


In [26]:
org_processing = org_processing[org_processing['country_code'] == 'USA']
len(org_processing)

15178

In [27]:
kaufmann_index_df = pd.read_csv(R"d:\msc-project\from-github\dissertations-2021-info\rahim_ghani_predict\data\kauffman\Kauffman_Indicators_Early-Stage_Entrepreneurship_Data_2020_v2_edited.csv",encoding='utf-8')

In [28]:
kaufmann_index_df.head(3)

,name,year,rne
0,Alabama,1998,0.00230
1,Alabama,1999,0.00206
2,Alabama,2000,0.00193


In [29]:
import us

state = us.states.lookup('alabama')
state.abbr

'AL'

In [30]:
list(kaufmann_index_df['name'])

['Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alabama',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Alaska',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arizona',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',
 'Arkansas',


In [31]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [32]:
kaufmann_index_df['state_code'] = kaufmann_index_df['name'].map(us_state_abbrev)
kaufmann_index_df['state_code'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', nan], dtype=object)

In [33]:
kaufmann_index_df.dropna(subset=['state_code'],inplace=True)

In [34]:
kaufmann_index_df['state_code'].unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'], dtype=object)

In [35]:
kaufmann_index_df.head(3)

,name,year,rne,state_code
0,Alabama,1998,0.00230,AL
1,Alabama,1999,0.00206,AL
2,Alabama,2000,0.00193,AL


In [36]:
kaufmann_index_df = kaufmann_index_df[['state_code','year','rne']]

In [37]:
kaufmann_index_df.head(3)

,state_code,year,rne
0,AL,1998,0.00230
1,AL,1999,0.00206
2,AL,2000,0.00193


In [38]:
org_processing.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year
3,46e0c7e4-64a3-f67e-2be2-6e707d6d5a51,2000-03-01,USA,NH,2000
5,1b923b7c-b415-a83c-cbcc-53b27c877907,2001-01-01,USA,IL,2001
6,88206b5f-eefe-e06c-5a84-0dedd98dd529,2001-01-01,USA,CA,2001


In [39]:
org_processing_v1 = pd.merge(org_processing,
                             kaufmann_index_df,
                             left_on=['state_code','founded_year'],
                             right_on=['state_code','year'],
                             how='left')

In [40]:
org_processing_v1['rne'] = (org_processing_v1['rne']*1000).astype(float).round(1)
org_processing_v1.head(3)

,org_uuid,founded_on,country_code,state_code,founded_year,year,rne
0,46e0c7e4-64a3-f67e-2be2-6e707d6d5a51,2000-03-01,USA,NH,2000,2000,2.6
1,1b923b7c-b415-a83c-cbcc-53b27c877907,2001-01-01,USA,IL,2001,2001,2.4
2,88206b5f-eefe-e06c-5a84-0dedd98dd529,2001-01-01,USA,CA,2001,2001,3.1


In [41]:
org_processing_v1.rename(columns={'rne':'kauffman_index'},inplace=True)

In [42]:
org_processing_v1['kauffman_index'].value_counts()

4.4    2699
3.6    2015
4.2    1900
4.0     659
2.8     595
2.9     585
2.6     582
3.4     581
3.5     568
3.2     482
2.5     462
2.2     424
2.3     371
3.9     364
2.4     356
3.1     344
2.7     296
3.7     205
3.8     204
1.8     194
4.3     185
2.0     183
3.0     161
1.9     147
3.3     137
2.1     128
1.6     117
1.7      99
4.7      41
1.4      28
4.1      24
5.0      22
4.5       7
4.9       5
4.6       3
5.1       2
1.5       2
4.8       1
Name: kauffman_index, dtype: int64

In [43]:
main_processing_df_v2 = pd.merge(main_processing_df_v1,
                                 org_processing_v1[['org_uuid','kauffman_index']],
                                 on='org_uuid',
                                 how='left')
main_processing_df_v2.head(3)

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video,kauffman_index
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,NaN
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN


In [44]:
main_processing_df_v2['kauffman_index'].fillna(0,inplace=True)

In [45]:
main_processing_df_v2.head(10)

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video,kauffman_index
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
3,46e0c7e4-64a3-f67e-2be2-6e707d6d5a51,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,2.6
4,89d81ccd-1b5d-4adb-1bcc-312ab6e4f90d,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
5,1b923b7c-b415-a83c-cbcc-53b27c877907,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,2.4
6,88206b5f-eefe-e06c-5a84-0dedd98dd529,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.1
7,91f5ca18-e026-9482-8e4f-22593943e0f5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3.1
8,3d607a21-39bb-651e-c53f-9890ee334f18,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,3.0
9,ab272c7d-735f-8ff7-d5b6-009ac6d91792,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.3


In [46]:
#one-hot encode for kauffman index

#one_hot_kauffman_index = pd.get_dummies(main_processing_df_v2['kauffman_index'])

#create column name
#column_name = list()
#for column in list(pd.get_dummies(one_hot_kauffman_index.columns)):
    #column_name.append(str('is_kauffman_index_' + str(column)))

#new_column_dict = dict(zip(one_hot_kauffman_index.columns,column_name))
#one_hot_kauffman_index.rename(columns = new_column_dict,inplace=True)
#one_hot_kauffman_index.head(3)

In [47]:
#one_hot_kauffman_index.shape

In [48]:
#main_processing_df_v3=pd.concat([main_processing_df_v2,
                                 #one_hot_kauffman_index],
                                #axis=1)

In [49]:
#main_processing_df_v3.head(3)

In [50]:
#main_processing_df_v3.drop(columns = ['kauffman_index'],inplace=True)

In [51]:
main_processing_df_v2.head(5)

,org_uuid,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,...,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video,kauffman_index
0,eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
1,0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
2,de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0
3,46e0c7e4-64a3-f67e-2be2-6e707d6d5a51,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,2.6
4,89d81ccd-1b5d-4adb-1bcc-312ab6e4f90d,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0


In [52]:
len(main_processing_df_v2)

28658

# Rename columns and save final file

In [53]:
main_processing_df_v2.set_index('org_uuid',inplace=True)

In [54]:
main_processing_df_v2.head(3)

,is_administrative_services,is_advertising,is_agriculture_and_farming,is_apps,is_artificial_intelligence,is_biotechnology,is_clothing_and_apparel,is_commerce_and_shopping,is_community_and_lifestyle,is_consumer_electronics,...,is_real_estate,is_sales_and_marketing,is_science_and_engineering,is_software,is_sports,is_sustainability,is_transportation,is_travel_and_tourism,is_video,kauffman_index
org_uuid,,,,,,,,,,,,,,,,,,,,,
eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0


In [55]:
new_column_name = list()
for column in list(main_processing_df_v2.columns):
    new_column_name.append('innovation_'+column)

In [58]:
new_column_dict = dict(zip(main_processing_df_v2.columns,new_column_name))
main_processing_df_v2.rename(columns = new_column_dict,inplace=True)

In [59]:
main_processing_df_v2.head(3)

,innovation_is_administrative_services,innovation_is_advertising,innovation_is_agriculture_and_farming,innovation_is_apps,innovation_is_artificial_intelligence,innovation_is_biotechnology,innovation_is_clothing_and_apparel,innovation_is_commerce_and_shopping,innovation_is_community_and_lifestyle,innovation_is_consumer_electronics,...,innovation_is_real_estate,innovation_is_sales_and_marketing,innovation_is_science_and_engineering,innovation_is_software,innovation_is_sports,innovation_is_sustainability,innovation_is_transportation,innovation_is_travel_and_tourism,innovation_is_video,innovation_kauffman_index
org_uuid,,,,,,,,,,,,,,,,,,,,,
eb37f7c8-c036-a915-bb7c-63f445330f66,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
0a88fa0e-7abe-c4ac-ab69-877b01a9cb60,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0.0
de462c42-b0a5-c5af-9637-ec426b4e991f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0.0


In [60]:
main_processing_df_v2.to_csv(r'd:\msc-project\data\final\features_innovation.csv',encoding='utf=8')